# Model building for Dog Breed Classifier

### Here we use tensorflow transfer learning technique. We are using ' Inception ResNet V2' model for giving features and at last we are building feed forward networks for identifying the breed based on features.

### Loading all modules

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from glob import glob
dog_names = [item[15:-1] for item in sorted(glob("/content/train/*/"))]
print(dog_names)
labels = {idx:item[15:-1] for idx, item in enumerate(sorted(glob("/content/train/*/")))}
print(labels)

['affenpinscher', 'afghan_hound', 'african_hunting_dog', 'airedale', 'american_staffordshire_terrier', 'appenzeller', 'australian_terrier', 'basenji', 'basset', 'beagle', 'bedlington_terrier', 'bernese_mountain_dog', 'black-and-tan_coonhound', 'blenheim_spaniel', 'bloodhound', 'bluetick', 'border_collie', 'border_terrier', 'borzoi', 'boston_bull', 'bouvier_des_flandres', 'boxer', 'brabancon_griffon', 'briard', 'brittany_spaniel', 'bull_mastiff', 'cairn', 'cardigan', 'chesapeake_bay_retriever', 'chihuahua', 'chow', 'clumber', 'cocker_spaniel', 'collie', 'curly-coated_retriever', 'dandie_dinmont', 'dhole', 'dingo', 'doberman', 'english_foxhound', 'english_setter', 'english_springer', 'entlebucher', 'eskimo_dog', 'flat-coated_retriever', 'french_bulldog', 'german_shepherd', 'german_short-haired_pointer', 'giant_schnauzer', 'golden_retriever', 'gordon_setter', 'great_dane', 'great_pyrenees', 'greater_swiss_mountain_dog', 'groenendael', 'ibizan_hound', 'irish_setter', 'irish_terrier', 'iris

In [2]:
%cd /content/drive/MyDrive/DataScience/dog-breed-classifier
!pip install -r requirements.txt

/content/drive/.shortcut-targets-by-id/1yGnr-2ym6Be72K-0nm_p7q9CJlhbXZaD/DataScience/dog-breed-classifier
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 5.0 MB/s 
     |████████████████████████████████| 156 kB 32.6 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 114 kB 67.0 MB/s 
     |████████████████████████████████| 3.8 MB 59.8 MB/s 
     |████████████████████████████████| 77 kB 7.5 MB/s 
     |████████████████████████████████| 2.9 MB 48.4 MB/s 
     |████████████████████████████████| 125 kB 72.9 MB/s 
     |████████████████████████████████| 96 kB 6.6 MB/s 
     |████████████████████████████████| 147 kB 64.7 MB/s 
     |████████████████████████████████| 9.5 MB 50.5 MB/s 
     |████████████████████████████████| 2.2 MB 44.6 MB/s 
     |████████████████████████████████| 1.0 MB 33.9 MB/s 
     |████████████████████████████████| 227 kB 7

In [3]:
import tensorflow
tensorflow.__version__

'2.3.1'

In [6]:
!nvidia-smi

Thu Nov 17 05:23:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model,load_model
from glob import glob
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,Activation,GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
#Execute below 2 lines if you are using NVIDIA GPU or else comment it
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0],True)

### We load model without top for our purpose

In [8]:
Inceptionv4_model = InceptionResNetV2(include_top=False,input_shape=[299,299,3])

219062272/219055592 [==============================] - 1s 0us/step


### As the transfer learning models are trained on huge data we need not train it again.

In [9]:
for layer in Inceptionv4_model.layers:
    layer.trainable = False

### Importent here is to note that I had used GlobalAveragePooling2D() instead of Flatten(). Reason here is simple:
### <li>I wanted to deploy the model. So to keep its size small I used GlobalAverage instead of Flatten.
###  Because, GlobalAverage drops many parameters where Flatten takes all of them. Since Im having transfer learning model, Its ok to loose some of the information and it doesnt effect much.
#### (Purely on experiment basis)

In [10]:
x =GlobalAveragePooling2D()(Inceptionv4_model.output)

x = Dense(256)(x)

x = Dropout(0.4)(x)

x = Dense(120)(x)

x = Dropout(0.4)(x)

pred = Activation('softmax')(x)

### Joining tranfer learning model with Dense layers

In [11]:
model = Model(inputs=Inceptionv4_model.input,outputs=pred)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

## Generation of training and test data
### Here we increase the training data by data augumentation process

In [12]:
%cd /content/
!unzip /content/drive/MyDrive/DataScience/dog-breed-classifier/Data/train.zip
!unzip /content/drive/MyDrive/DataScience/dog-breed-classifier/Data/test.zip
root = "/content/train/"
root1 = "/content/test/"

Streaming output truncated to the last 5000 lines.
  inflating: train/siberian_husky/2db83abf17de07286072ee5a2d497830.jpg  
  inflating: train/siberian_husky/77928a7be51832f25f6909eb9cc4afce.jpg  
  inflating: train/siberian_husky/b19813c11f94f561e45242347d8da73a.jpg  
  inflating: train/siberian_husky/77b43f8c92647abe6025d4972dd899cf.jpg  
  inflating: train/siberian_husky/219a919735822e98d89cca96fd89107b.jpg  
  inflating: train/siberian_husky/351f02bbaa4f24bad17cc1e83c39a0fc.jpg  
  inflating: train/siberian_husky/b3b5eb135bc850b54a04d2c43b83a237.jpg  
  inflating: train/siberian_husky/1639ccef237eec9fc4eb1aa0916201c3.jpg  
  inflating: train/siberian_husky/bc198e31d0c17f265239d4cf515104cf.jpg  
  inflating: train/siberian_husky/f3afd4f69038b249042cd481f0b5813a.jpg  
  inflating: train/siberian_husky/14597c96d5c222eebd742f4207296314.jpg  
  inflating: train/siberian_husky/247bd5917165aae21518dc8a5ac6d0ab.jpg  
  inflating: train/siberian_husky/0d74e47d162e023bb1ae5aae40276fd6.jpg  


In [13]:
train_data =  ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest'
                                   )
test_data = ImageDataGenerator(rescale=1./255)


train_set = train_data.flow_from_directory(root,
                                              target_size=(299,299),
                                              batch_size=16,
                                              class_mode='categorical'
                                             )

test_set = train_data.flow_from_directory(root1,
                                              target_size=(299,299),
                                              batch_size=16,
                                              class_mode='categorical'
                                             )

Found 7222 images belonging to 120 classes.
Found 3000 images belonging to 120 classes.


In [14]:
print(test_set.class_indices)

{'affenpinscher': 0, 'afghan_hound': 1, 'african_hunting_dog': 2, 'airedale': 3, 'american_staffordshire_terrier': 4, 'appenzeller': 5, 'australian_terrier': 6, 'basenji': 7, 'basset': 8, 'beagle': 9, 'bedlington_terrier': 10, 'bernese_mountain_dog': 11, 'black-and-tan_coonhound': 12, 'blenheim_spaniel': 13, 'bloodhound': 14, 'bluetick': 15, 'border_collie': 16, 'border_terrier': 17, 'borzoi': 18, 'boston_bull': 19, 'bouvier_des_flandres': 20, 'boxer': 21, 'brabancon_griffon': 22, 'briard': 23, 'brittany_spaniel': 24, 'bull_mastiff': 25, 'cairn': 26, 'cardigan': 27, 'chesapeake_bay_retriever': 28, 'chihuahua': 29, 'chow': 30, 'clumber': 31, 'cocker_spaniel': 32, 'collie': 33, 'curly-coated_retriever': 34, 'dandie_dinmont': 35, 'dhole': 36, 'dingo': 37, 'doberman': 38, 'english_foxhound': 39, 'english_setter': 40, 'english_springer': 41, 'entlebucher': 42, 'eskimo_dog': 43, 'flat-coated_retriever': 44, 'french_bulldog': 45, 'german_shepherd': 46, 'german_short-haired_pointer': 47, 'gian

### Creating the checkpoint of the best model

In [15]:
checkpoint = ModelCheckpoint('/content/drive/MyDrive/DataScience/dog-breed-classifier/saved_models/Inceptionv4_model.h5',monitor='val_accuracy',verbose=1,save_best_only=True)
callback = [checkpoint]

### Compiling of model with Adam optimizer and crossentropy loss

In [16]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

### Training process

In [22]:
history = model.fit(train_set,epochs=30,steps_per_epoch=len(train_set),validation_data=test_set,verbose=1,callbacks=callback)

AttributeError: ignored

In [21]:
!pip uninstall Pillow
!pip install Pillow==9.0.0

Found existing installation: Pillow 9.3.0
Uninstalling Pillow-9.3.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/PIL/*
    /usr/local/lib/python3.7/dist-packages/Pillow-9.3.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/Pillow.libs/libXau-00ec42fe.so.6.0.0
    /usr/local/lib/python3.7/dist-packages/Pillow.libs/libbrotlicommon-cf2297e4.so.1.0.9
    /usr/local/lib/python3.7/dist-packages/Pillow.libs/libbrotlidec-97e69943.so.1.0.9
    /usr/local/lib/python3.7/dist-packages/Pillow.libs/libfreetype-7d9be1ab.so.6.18.3
    /usr/local/lib/python3.7/dist-packages/Pillow.libs/libharfbuzz-5e08a948.so.0.50301.0
    /usr/local/lib/python3.7/dist-packages/Pillow.libs/libjpeg-b1f3a3b7.so.62.3.0
    /usr/local/lib/python3.7/dist-packages/Pillow.libs/liblcms2-1e643a89.so.2.0.13
    /usr/local/lib/python3.7/dist-packages/Pillow.libs/liblzma-816f5b19.so.5.2.7
    /usr/local/lib/python3.7/dist-packages/Pillow.libs/libopenjp2-fca9bf24.so.2.5.0
    /usr/local/lib/python3.7/dist-pack

In [24]:
!python --version

Python 3.7.15


## At the end we got 52.4% training accuracy and 94% accuracy on test data
#  <center>Thank You